In [1]:
import numpy as np
import pandas as pd
from quetzal.model import stepmodel

C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\pyproj\crs.py:422: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  projstring = _prepare_from_string(projparams)


# Modelling steps 3 and 4.
## Needs a network, clustered zones with volumes and level of service attributes.

In [2]:
input_path = '../input/estimation/'
output_path = '../output/'
model_path = '../model/'

In [3]:
sm = stepmodel.read_zip(model_path + 'de_step_3.zip')

zones: 100%|███████████████████████████████████████████████████████████████████████████| 36/36 [00:31<00:00,  1.13it/s]


## Assignment

In [4]:
# Make path to tuples as workaround for pandas TypeError
# (see issue: https://github.com/pandas-dev/pandas/issues/31177)
sm.los['path'] = sm.los['path'].apply(lambda p: tuple(p))

In [5]:
# Add probabilities to the LoS table
sm.pt_los = sm.pt_los.merge(
    sm.probabilities.set_index(['origin', 'destination', 'segment'])[
        ['walk', 'car', 'air', 'bus', 'coach', 'rail_short_distance', 'rail_long_distance']
    ].stack().rename('probability').unstack('segment'),
    how='left',
    on=['origin', 'destination', 'route_type']
).rename(columns={seg: (seg, 'probability') for seg in sm.segments})
sm.pt_los.shape

(654107, 21)

In [6]:
sm.pt_los.sample(n=3)

,origin,destination,path,link_path,access_time,footpath_time,in_vehicle_time,waiting_time,boarding_time,time,...,length,route_types,route_type,ntransfers,accessibility,"(Fz1, probability)","(Fz2, probability)","(Fz3, probability)","(Fz4, probability)","(Fz6, probability)"
444412,DEA38,DE144,"(DEA38, DEA38_rail_short_distance_0, DEA38_bus...","[bus_598375, bus_303613, bus_303626, rail_long...",1866.878699,227.622312,16380.0,1800.0,720,20994.501012,...,642999.668865,"(bus, rail_long_distance)",rail_long_distance,5,0.168453,0.715201,5.345511e-04,5.274296e-06,0.003269,0.000257
519076,DEF07,DEG0A,"(DEF07, DEF07_bus_4, DEF07_rail_short_distance...","[rail_short_21049, rail_short_16026, rail_shor...",982.854295,147.204928,14100.0,3000.0,1200,19430.059223,...,640506.362630,"(rail_short_distance,)",rail_short_distance,9,0.040215,0.866634,6.662725e-07,7.386238e-09,0.000203,0.000004
489838,DE737,DE25C,"(DE737, DE737_rail_short_distance_1, rail_shor...","[rail_short_21504, rail_short_19224, rail_shor...",1581.894200,122.055452,7620.0,2100.0,840,12263.949652,...,310213.651943,"(rail_short_distance, bus, rail_long_distance)",rail_long_distance,6,0.047358,0.748667,9.850905e-02,5.146391e-04,0.020501,0.004921


In [8]:
sm.step_pt_assignment(
    volume_column=None,
    road=False,
    od_stack=True) # Output DataFrame

Fz6: 100%|███████████████████████████████████████████████████████████████████████████████| 5/5 [01:26<00:00, 17.35s/it]


In [9]:
sm.car_los = sm.los.loc[sm.los['route_type']=='car']
sm.step_car_assignment()

Fz6: 100%|███████████████████████████████████████████████████████████████████████████████| 5/5 [02:27<00:00, 29.46s/it]


## Results

In [21]:
sm.loaded_links.sample(n=3)

,geometry,route_id,route_type,a,b,time,trip_id,link_sequence,length,headway,price,load
rail_short_430,"LINESTRING (11.10754 47.95082, 10.95955 48.30092)",rail_short_28,rail_short_distance,DE21E_rail_short_distance_2,DE275_rail_short_distance_0,3000.0,rail_short_1118,3.0,41612.741185,600,13.597480,3449.396237
bus_253794,"LINESTRING (6.96008 51.53384, 6.93625 51.50983)",bus_12591,bus,DEA31_bus_8,DEA31_bus_9,360.0,bus_436762,4.0,3739.820599,600,0.000000,0.000000
rail_short_19429,"LINESTRING (10.20855 51.39353, 10.58582 51.12075)",rail_short_58,rail_short_distance,DEG06_rail_short_distance_2,DEG09_rail_short_distance_2,2040.0,rail_short_44803,1.0,51319.048258,600,15.746898,0.000000


In [16]:
sm.road_links.sample(n=3)

,a,b,length,geometry,highway,maxspeed,lanes,time,index,"(Fz1, car)","(Fz2, car)","(Fz3, car)","(Fz4, car)","(Fz6, car)","(all, car)"
id,,,,,,,,,,,,,,,
rl_268898,30836839,1637700551,157.289,"LINESTRING (8.60267 49.98363, 8.60059 49.98316)",primary,70,3,10.111436,rl_268898,724331.959907,39938.131463,555031.988897,905745.062171,2.013295e+06,4.238342e+06
rl_190777,5356951507,5356951500,5.181,"LINESTRING (6.65503 49.68506, 6.65498 49.68509)",primary,100,3,0.233145,rl_190777,NaN,NaN,NaN,NaN,NaN,0.000000e+00
rl_74553,114412629,3665739031,535.307,"LINESTRING (10.78373 51.83774, 10.77762 51.83609)",primary,50,"[3, 2]",48.177630,rl_74553,NaN,NaN,NaN,NaN,NaN,0.000000e+00


### Excel export

In [44]:
stacks = {name[6:]: attr for name, attr in sm.__dict__.items() if name[:5] == 'stack'}
with pd.ExcelWriter(output_path + 'stacks.xlsx') as writer:  
    for name, stack in stacks.items():
        try:
            stack.reset_index().to_excel(writer, sheet_name=name, index=False)
        except ValueError: # DataFrame too large for Excel
            print('Could not save sheet ' + name)

Could not save sheet link_sum


### Save model

In [17]:
sm.__dict__.keys()

dict_keys(['parameters', 'debug', 'walk_on_road', 'coordinates_unit', 'epsg', 'segments', 'checkpoint_links', 'loaded_links', 'links', 'road_links', 'footpaths', 'lines', 'networkcaster_neighbors', 'networkcaster_road_access', 'node_parenthood', 'road_to_transit', 'zone_to_transit', 'zone_to_road', 'checkpoint_nodes', 'loaded_nodes', 'nodes', 'road_nodes', 'centroids', 'disaggregated_nodes', 'micro_zones', 'zones', 'node_clusters', 'fare_rules', 'fare_attributes', 'mode_nests', 'logit_scales', 'utility_values', 'agencies', 'car_los', 'jsons', 'los', 'mode_utility', 'probabilities', 'pt_los', 'pt_routes', 'utilities', 'volumes', 'dead_ends', 'pt_route_types', 'broken_sequences', 'od_probabilities', 'od_utilities'])

In [18]:
sm.to_zip(model_path + r'de_step_4.zip')

volumes: 100%|█████████████████████████████████████████████████████████████████████████| 36/36 [01:24<00:00,  2.35s/it]
